In [1]:
# This snippet of code properly adds the working source root path to python's path
# so you no longer have to install spykshrk through setuptools
import sys, os 
root_depth = 2
notebook_dir = globals()['_dh'][0]
root_path = os.path.abspath(os.path.join(notebook_dir, '../'*root_depth))
# Add to python's path
try:
    while True:
        sys.path.remove(root_path)
except ValueError:
    # no more root paths
    pass
sys.path.append(root_path)
# Alternatively set root path as current working directory
#os.chdir(root_path)


In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import functools
import itertools
import holoviews as hv
from holoviews import dim, opts
import enum
import IPython
import warnings
import torch
import collections

from IPython.utils.text import columnize
from IPython.lib.pretty import pprint, pretty
from IPython.display import display_html

# Spykshrk modules for data analysis
from spykshrk.franklab.data_containers import FlatLinearPosition, SpikeFeatures, \
        EncodeSettings, pos_col_format, SpikeObservation
from spykshrk.franklab.generator.place.spike_generator import UnitNormalGenerator, TetrodeUniformUnitNormalGenerator
from spykshrk.franklab.generator.place.pos_generator import WtrackPosConstSimulator
from spykshrk.franklab.pp_decoder.util import normal_pdf_int_lookup, gaussian
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPEncoder, OfflinePPDecoder
from spykshrk.franklab.pp_decoder.wtrack_mapping import WtrackLinposDecomposer, WtrackLinposRecomposer
from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer, WtrackLinposVisualizer
from spykshrk.franklab.visualization import LinPosVisualizer, TetrodeVisualizer
from spykshrk.franklab.pp_decoder.util import apply_no_anim_boundary
from spykshrk.franklab.wtrack import WtrackArm, Direction, Order, Rotation
from spykshrk.util import AttrDict, AttrDictEnum

hv.extension('bokeh')
hv.extension('matplotlib')

hv.renderer('bokeh').theme = "dark_minimal"
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'
plt.rcParams['figure.constrained_layout.use'] = False
plt.rcParams['xtick.major.pad'] = 3.5
plt.rcParams['ytick.major.pad'] = 10
plt.rcParams['axes.formatter.limits'] = [-5,5]

pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)

plain_formatter = get_ipython().display_formatter.formatters['text/plain']
plain_formatter.max_width=160
plain_formatter.max_seq_length=20

np.set_printoptions(precision=4, linewidth=80, threshold=20, edgeitems=5)

In [5]:
class InconsistentDataWarning(Warning):
    pass

In [6]:
# Encoding and decoding settings for both simulator and algorithm

encode_settings = AttrDict({'sampling_rate': 1000,
                            'pos_sampling_rate': 30,
                            'pos_bins': np.arange(0,800,1),
                            'pos_bin_edges': np.arange(0,800.1,1),
                            'pos_bin_delta': 1,
                            'pos_kernel': sp.stats.norm.pdf(np.arange(0,800,1), 400, 2),
                            'pos_kernel_std': 2, 
                            'mark_kernel_std': int(10), 
                            'pos_num_bins': 800,
                            'pos_col_names': [pos_col_format(ii, 800) for ii in range(800)],
                            'arm_coordinates': [[0, 100], [100, 250], [250,400], [400, 500], [500, 650], [650, 800]],
                            'wtrack_arm_coordinates': AttrDictEnum({WtrackArm.center: 
                                                                    AttrDictEnum({Direction.outbound: AttrDictEnum(x1=0, x2=100, len=100), 
                                                                                  Direction.inbound: AttrDictEnum(x1=400, x2=500, len=100)}),
                                                                    WtrackArm.left: 
                                                                    AttrDictEnum({Direction.outbound: AttrDictEnum(x1=100, x2=250, len=150),
                                                                                  Direction.inbound: AttrDictEnum(x1=250, x2=400, len=150)}),
                                                                    WtrackArm.right: 
                                                                    AttrDictEnum({Direction.outbound: AttrDictEnum(x1=500, x2=650, len=150),
                                                                                  Direction.inbound: AttrDictEnum(x1=650, x2=800, len=150)})}),
                            'vel': 3,
                            'spk_amp': 60})

decode_settings = AttrDict({'trans_smooth_std': 5,
                            'trans_uniform_gain': 0.001,
                            'time_bin_size': 10})
                            

In [7]:
wtrack_pos_const_sim = WtrackPosConstSimulator(trial_time=10, num_series=3, encode_settings=encode_settings)
linpos_flat = wtrack_pos_const_sim.linpos_flat

In [8]:
hv.output(backend='matplotlib')
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'

wtrack_viz = WtrackLinposVisualizer(linpos_flat, encode_settings)
wtrack_viz.plot

In [9]:
wtrack_decomposed = WtrackLinposDecomposer(linpos_flat, encode_settings)

In [45]:
wtrack_decomposed.decomp_linpos

In [10]:
tet_gen = TetrodeUniformUnitNormalGenerator(sampling_rate=encode_settings.sampling_rate,
                                            num_marks=4,
                                            num_units=200,
                                            mark_mean_range=(60, 600),
                                            mark_cov_range=(50, 120),
                                            firing_rate_range=(5, 40),
                                            pos_field_range=wtrack_decomposed.simple_main_armcoord_cw,
                                            pos_field_bins=wtrack_decomposed.simple_main_armcoord_bins_cw,
                                            pos_field_var_range=(2,10))

In [11]:
resample_decomp_linpos = wtrack_decomposed.decomp_linpos.get_resampled(1)

spk_amps, unit_spks = tet_gen.simulate_tetrode_over_pos(resample_decomp_linpos, col_name='linpos_cw')
spk_amps_prev, unit_spks_prev = tet_gen.simulate_tetrode_over_pos(resample_decomp_linpos, col_name='linpos_cw_prev')
spk_amps_next, unit_spks_next = tet_gen.simulate_tetrode_over_pos(resample_decomp_linpos, col_name='linpos_cw_next')

In [12]:
spk_amps = spk_amps.append([spk_amps_prev, spk_amps_next]).sort_index(level='timestamp')

In [13]:

tet_viz = TetrodeVisualizer(spk_amps, linpos_flat.get_irregular_resampled(spk_amps), unit_spks)
#tet_viz.plot_color_3d_dynamic('linpos_cw', 'c00', 'c01')
tet_viz.plot_color_3d_dynamic('linpos_flat', 'c00', 'c01')

In [14]:
spks_linpos = linpos_flat.get_irregular_resampled(spk_amps).join(spk_amps)
spk_linpos_outbound = spks_linpos.query('direction==@Direction.outbound', engine='python')
spk_linpos_inbound = spks_linpos.query('direction==@Direction.inbound', engine='python')

In [48]:
class WtrackPPInOutEncoder:
    
    def __init__(self, wtrack_decomposed, enc_spk_amps, dec_spk_amps, encode_settings, decode_settings, chunk_size, cuda, norm):
        self.wtrack_decomposed = wtrack_decomposed
        self.enc_spk_amps = enc_spk_amps
        self.dec_spk_amps = dec_spk_amps
        self.encode_settings = encode_settings
        self.decode_settings = decode_settings
        self.chunk_size = chunk_size
        self.cuda = cuda
        self.norm = norm
        
        self.recomposed_encoder_in = None
        self.recomposed_encoder_out = None
                
        self.spks_linpos = self.wtrack_decomposed.decomp_linpos.get_irregular_resampled(self.enc_spk_amps).join(self.enc_spk_amps)
        self.spks_linpos_outbound = self.spks_linpos.query('direction==@Direction.outbound', engine='python')
        self.spks_linpos_outbound = self.spks_linpos_outbound.drop(columns=self.wtrack_decomposed.decomp_linpos.columns)
        self.spks_linpos_inbound = self.spks_linpos.query('direction==@Direction.inbound', engine='python')
        self.spks_linpos_inbound = self.spks_linpos_inbound.drop(columns=self.wtrack_decomposed.decomp_linpos.columns)
        
    def run_encoder(self):
        self.recomposed_encoder_out = WtrackPPInOutEncoder.create_recomposed_encoder(decomp_linpos = self.wtrack_decomposed.decomp_linpos, 
                                                                                     enc_spk_amp = self.spks_linpos_outbound,
                                                                                     dec_spk_amp = self.dec_spk_amps,
                                                                                     wtrack_decomposed = self.wtrack_decomposed,
                                                                                     encode_settings = self.encode_settings,
                                                                                     decode_settings = self.decode_settings,
                                                                                     chunk_size = self.chunk_size,
                                                                                     cuda = self.cuda,
                                                                                     norm = self.norm)
        
        self.recomposed_encoder_in = WtrackPPInOutEncoder.create_recomposed_encoder(decomp_linpos = self.wtrack_decomposed.decomp_linpos, 
                                                                                    enc_spk_amp = self.spks_linpos_inbound,
                                                                                    dec_spk_amp = self.dec_spk_amps,
                                                                                    wtrack_decomposed = self.wtrack_decomposed,
                                                                                    encode_settings = self.encode_settings,
                                                                                    decode_settings = self.decode_settings,
                                                                                    chunk_size = self.chunk_size,
                                                                                    cuda = self.cuda,
                                                                                    norm = self.norm)
        
    @staticmethod
    def create_recomposed_encoder(decomp_linpos, enc_spk_amp, dec_spk_amp, wtrack_decomposed, encode_settings, decode_settings, chunk_size=10000, cuda=True, norm=True):
        encoder_cw = OfflinePPEncoder(linflat=decomp_linpos, enc_spk_amp=enc_spk_amp, dec_spk_amp=dec_spk_amp, 
                                      encode_settings=wtrack_decomposed.encode_settings_decomp, decode_settings=decode_settings,
                                      linflat_col_name='linpos_cw', chunk_size=chunk_size, cuda=cuda, norm=norm)
        encoder_ccw = OfflinePPEncoder(linflat=decomp_linpos, enc_spk_amp=enc_spk_amp, dec_spk_amp=dec_spk_amp, 
                                       encode_settings=wtrack_decomposed.encode_settings_decomp, decode_settings=decode_settings,
                                       linflat_col_name  ='linpos_ccw', chunk_size=chunk_size, cuda=cuda, norm=norm)
        observ_cw = encoder_cw.run_encoder()
        observ_ccw = encoder_ccw.run_encoder()
        
        recomposed = WtrackLinposRecomposer(encoder_cw, encoder_ccw, wtrack_decomposed, encode_settings)
        
        return recomposed
        

In [49]:
%%time

multi_encoder = WtrackPPInOutEncoder(wtrack_decomposed, spk_amps, spk_amps, encode_settings, decode_settings, 15000, cuda=False, norm=False)


In [50]:
%%time

multi_encoder.run_encoder()

In [53]:
hv.output(backend='matplotlib')

observ_ex = multi_encoder.recomposed_encoder_in.encoder_cw.observ_obj
encode_settings_decomp = multi_encoder.wtrack_decomposed.encode_settings_decomp

# Setup plot to visualize estimated position distribution
sel_distrib = observ_ex.loc[:, pos_col_format(0,encode_settings_decomp.pos_num_bins):         
                            pos_col_format(encode_settings_decomp.pos_num_bins-1,
                                           encode_settings_decomp.pos_num_bins)]
    
sel_pos = observ_ex.loc[:, 'position']
max_prob = sel_distrib.max().max()

num_plot = 20

def plot_observ(ind):
    plot_list = []
    for ii in range(num_plot):
        sel = sel_distrib.iloc[ind+ii]
        plot_list.append(hv.Curve(sel))
        plot_list.append(hv.Points((sel_pos.iloc[ind+ii], [sel.max()/10])))
    return hv.Overlay(plot_list).opts(fig_size=300, aspect=3)
        
dmap = hv.DynamicMap(plot_observ, kdims=['ind'])
dmap.redim.values(ind=list(range(0, len(observ_ex)-num_plot, num_plot)))

In [21]:
class ForwardReverseDecoder:
    
    def __init__(self, observ, trans_mat, prob_no_spike, encode_settings, decode_settings, time_bin_size):
        self.observ = observ
        self.trans_mat = trans_mat
        self.prob_no_spike = prob_no_spike
        self.encode_settings = encode_settings
        self.decode_settings = decode_settings
        self.time_bin_size = time_bin_size
        
        self.posterior_forward = None
        self.posterior_reverse = None
        
        self.trans_forward = np.triu(self.trans_mat)
        self.trans_reverse = np.tril(self.trans_mat)
        
        self.decoder_forward = OfflinePPDecoder(observ_obj=self.observ, trans_mat=self.trans_forward, 
                                                prob_no_spike=self.prob_no_spike, encode_settings=self.encode_settings,
                                                decode_settings=self.decode_settings, time_bin_size=self.time_bin_size)
        self.decoder_reverse = OfflinePPDecoder(observ_obj=self.observ, trans_mat=self.trans_reverse, 
                                                prob_no_spike=self.prob_no_spike, encode_settings=self.encode_settings,
                                                decode_settings=self.decode_settings, time_bin_size=self.time_bin_size)
    
    def run_decoder(self):
        self.posterior_forward = self.decoder_forward.run_decoder()
        self.posterior_reverse = self.decoder_reverse.run_decoder()


In [22]:
time_bin_size = 10

decoder = ForwardReverseDecoder(observ=multi_encoder.recomposed_encoder_in.observ, trans_mat=multi_encoder.recomposed_encoder_in.trans_mat,
                                prob_no_spike=multi_encoder.recomposed_encoder_in.prob_no_spike, encode_settings=multi_encoder.encode_settings,
                                decode_settings=decode_settings, time_bin_size=time_bin_size)
decoder.run_decoder()

In [23]:
%%time
# Run PP decoding algorithm
time_bin_size = 10

decoder_in = OfflinePPDecoder(observ_obj=multi_encoder.recomposed_encoder_in.observ, trans_mat=multi_encoder.recomposed_encoder_in.trans_mat, 
                              prob_no_spike=multi_encoder.recomposed_encoder_in.prob_no_spike,
                              encode_settings=encode_settings, decode_settings=decode_settings, 
                              time_bin_size=time_bin_size, dtype=np.float32)

decoder_out = OfflinePPDecoder(observ_obj=multi_encoder.recomposed_encoder_out.observ, trans_mat=multi_encoder.recomposed_encoder_out.trans_mat, 
                               prob_no_spike=multi_encoder.recomposed_encoder_out.prob_no_spike,
                               encode_settings=encode_settings, decode_settings=decode_settings, 
                               time_bin_size=time_bin_size, dtype=np.float32)

posteriors_in = decoder_in.run_decoder()
posteriors_out = decoder_out.run_decoder()

In [24]:
hv.output(backend='bokeh', size=100)
hv.Curve(multi_encoder.recomposed_encoder_in.observ.loc[:, 'x000':'x799'].iloc[2].values)

In [25]:
hv.Curve(multi_encoder.recomposed_encoder_in.encoder_cw.occupancy)

In [26]:
hv.Curve(multi_encoder.recomposed_encoder_in.encoder_cw.firing_rate[1])

In [27]:
hv.Curve(multi_encoder.recomposed_encoder_in.encoder_cw.firing_rate[1]) * hv.Curve(multi_encoder.recomposed_encoder_in.encoder_cw.occupancy) * \
    hv.Curve((multi_encoder.recomposed_encoder_in.encoder_cw.firing_rate[1]+1) / (multi_encoder.recomposed_encoder_in.encoder_cw.occupancy))

In [28]:
(hv.Curve(multi_encoder.recomposed_encoder_in.encoder_cw.prob_no_spike[1]).opts(frame_width=800) + \
 hv.Curve(multi_encoder.recomposed_encoder_in.encoder_ccw.prob_no_spike[1]).opts(frame_width=800)).cols(1)

In [29]:
hv.Curve(multi_encoder.recomposed_encoder_in.prob_no_spike[1]).opts(frame_width=800)

In [30]:
display(decoder_in.likelihoods)

hv.Curve(decoder_in.likelihoods.loc[:, 'x000':'x799'].iloc[1].values).opts(frame_width=800)

In [31]:
#%%output backend='bokeh' size=300 holomap='scrubber'
#%%opts RGB { +framewise} [height=100 width=350]
#%%opts Points (marker='o' color='#AAAAFF' size=1 alpha=0.05)
#%%opts Polygons (color='grey', alpha=0.3 fill_color='grey' fill_alpha=0.3)
#%%opts Image {+framewise}
hv.output(backend='bokeh', size=200)

dec_viz = DecodeVisualizer(decoder_in.posteriors_obj, linpos=linpos_flat, enc_settings=encode_settings, heatmap_max=0.15)

dmap = dec_viz.plot_all_dynamic(stream=hv.streams.RangeXY())

dmap.opts(aspect=3, frame_width=600)

dmap

In [32]:
(multi_encoder.recomposed_encoder_in.trans_mat * decoder_in.posteriors_obj.get_distribution_view().iloc[2].values).sum()

In [33]:
(np.array([[0.25, 0.25], [0.1, 0.4]]) * np.array([0.5,0.5])).sum()

In [34]:
decoder_in.posteriors_obj.get_distribution_view().iloc[100].values.sum()

In [35]:
posteriors_in.get_distribution_view()